# 🎯 | Cora-For-Zava: 미세 조정을 통한 모델 맞춤화

환영합니다! 이 노트북에서는 지도 미세 조정(Supervised Fine-Tuning, SFT)을 사용하여 AI 모델을 맞춤화하고 특정 사용 사례에 맞춰 어조, 스타일 및 응답 일관성을 개선하는 방법을 안내합니다.

## 🛒 Zava 시나리오

**Cora**는 DIY 애호가를 위한 홈 인테리어 용품을 판매하는 가상의 소매업체 **Zava**의 고객 서비스 챗봇입니다. Cora의 응답을 개선하기 위해 소량의 예제(few-shot examples)와 RAG(Real-Assisted Gradient)를 사용해 왔지만, 이러한 접근 방식은 프롬프트 길이를 증가시켜 토큰 비용을 높이고 사용 가능한 컨텍스트 창을 줄입니다. 미세 조정을 통해 더 나은 어조와 스타일을 더 짧은 프롬프트에 직접 반영할 수 있습니다.


## 🎯 완성 목표

이 노트북을 마치면 다음을 수행하게 됩니다.
- ✅ 미세 조정을 위한 학습 데이터셋 준비 및 검증
- ✅ 토큰 사용량 분석을 통한 학습 효율성 최적화
- ✅ 학습 데이터를 Azure OpenAI에 업로드하여 처리
- ✅ 미세 조정 작업 제출 및 모니터링
- ✅ 미세 조정된 사용자 지정 모델 배포 및 테스트
- ✅ 개선된 어조 및 스타일 일관성 평가

## 💡 학습 목표

- 지도 학습 미세 조정을 위한 JSONL 데이터셋 준비 방법
- 토큰 개수 검증 및 학습 데이터 최적화 방법
- Azure OpenAI에서 미세 조정 작업 제출 및 모니터링 방법
- 미세 조정된 모델 배포 및 테스트 방법
- 미세 조정을 통해 프롬프트 길이를 줄이면서 일관성을 향상시키는 방법
- 미세 조정, few-shot prompting, RAG를 언제 사용해야 하는지

> **참고**: 미세 조정은 모델 동작을 기본 수준에서 사용자 지정하여 프롬프트 길이를 줄일 수 있도록 합니다. 품질과 일관성을 유지하면서 프롬프트를 표시합니다.

모델을 맞춤 설정할 준비가 되셨나요? 지금 시작해 보세요! 🚀

---

## Step 1: Verify Environment Variables

The following environment variables should already be configured in your `.env` file from the earlier setup steps:

- **AZURE_OPENAI_API_KEY**: Your Azure OpenAI API key
  - 제거하고 SystemManagedIdentity 사용 예정
- **AZURE_OPENAI_ENDPOINT**: Your Azure OpenAI service endpoint
- **AZURE_OPENAI_API_VERSION**: The API version to use (2025-02-01-preview for fine-tuning)
- **AZURE_SUBSCRIPTION_ID**: Your Azure subscription ID
- **AZURE_RESOURCE_GROUP**: Your Azure resource group name
- **AZURE_AI_PROJECT_NAME**: Your Azure AI Foundry project name

> **Important**: Fine-tuning requires specific API versions and model availability. Currently, `gpt-4o-2024-08-06` can be fine-tuned in Sweden Central and North Central US regions.

In [1]:
import os

openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_name = "gpt-4.1"
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-02-01-preview")

print("Using Model:", model_name)
print("Using API Version:", api_version)

Using Model: gpt-4.1
Using API Version: 2025-02-01-preview


## Step 2: 학습 데이터셋 준비

미세 조정을 위해서는 JSONL 형식으로 신중하게 준비된 학습 데이터가 필요합니다. 각 줄에는 원하는 어조와 스타일을 담은 대화 예시가 포함되어야 합니다. 저희 데이터셋은 다음과 같습니다.

- **학습 데이터셋** (`31-basic_training.jsonl`): 모델 학습용 예시
- **검증 데이터셋** (`31-basic_validation.jsonl`): 학습 중 성능 모니터링용 예시

> **주요 형식 요구 사항**:
> - 각 예시에는 대화 내용이 담긴 `messages` 배열이 있어야 합니다.
> - 메시지에는 `role` (시스템/사용자/어시스턴트)과 `content`가 포함되어야 합니다.
> - 학습 예시는 Zava의 원하는 어조(공손하고, 사실적이며, 도움이 되는)를 보여주어야 합니다.

In [2]:
# Identify Training and Validation datafiles


training_file = "./31-basic_training.jsonl"
validation_file = "./31-basic_validation.jsonl"

In [3]:
# Run preliminary checks

import json

# Load the training set
with open(training_file, 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open(validation_file, 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 40
First example in training set:
{'role': 'system', 'content': 'You are Cora, a polite, factual and helpful assistant for Zava, a DIY hardware store.'}
{'role': 'user', 'content': 'Is water-based polyurethane better than oil-based?'}
{'role': 'assistant', 'content': '🤔 Both have benefits! Water-based at $43 dries faster, oil-based at $34 gives amber tone. Which matters most?'}

Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'You are Cora, a polite, factual and helpful assistant for Zava, a DIY hardware store.'}
{'role': 'user', 'content': "What's good for painting wrought iron?"}
{'role': 'assistant', 'content': '🔧 Metal Primer is perfect for wrought iron! Metal Primer at $36 first, then any topcoat. Existing rust present?'}


## Step 3: 데이터 토큰 개수 분석

토큰 분석은 비용 추정 및 품질 향상에 매우 중요합니다. 다음 항목을 분석합니다.

- **총 토큰 수**: 시스템, 사용자 및 어시스턴트 메시지를 포함한 전체 대화 시간
- **어시스턴트 토큰 수**: 모델의 응답(개선 대상 학습 데이터)만 해당
- **분포 통계**: 최소/최대값, 평균/중앙값 및 백분위수 분석

> **모범 사례**:
> - 대화는 핵심적이고 간결하게 유지
> - 모든 예시에서 토큰 길이를 일정하게 유지
> - 비용 최적화를 위해 어시스턴트 토큰 비율 모니터링

In [4]:
# Validate token counts

import json
import tiktoken
import numpy as np

# default encoding for gpt-4o models. This requires the latest version of tiktoken to be installed.
encoding = tiktoken.get_encoding("o200k_base")


def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens


def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens


def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


files = [training_file, validation_file]

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: ./31-basic_training.jsonl

#### Distribution of total tokens:
min / max: 62, 78
mean / median: 68.375, 68.0
p5 / p95: 65.0, 71.1

#### Distribution of assistant tokens:
min / max: 19, 30
mean / median: 23.75, 24.0
p5 / p95: 20.9, 26.1
**************************************************
Processing file: ./31-basic_validation.jsonl

#### Distribution of total tokens:
min / max: 66, 76
mean / median: 69.4, 69.0
p5 / p95: 66.9, 73.3

#### Distribution of assistant tokens:
min / max: 22, 28
mean / median: 24.4, 24.0
p5 / p95: 22.9, 26.2
**************************************************


## Step 4: Azure에 미세 조정 데이터 업로드

준비된 데이터 세트를 Azure OpenAI에 업로드하여 처리합니다. 업로드된 파일은 다음과 같습니다.

- Azure OpenAI 리소스에 **안전하게 저장**
- 형식 준수 여부를 **자동으로 검증**
- 미세 조정 작업 생성 시 **접근 가능**
- Azure AI Foundry 포털의 '데이터 파일'에서 **확인 가능**

> **보안 참고**: 데이터는 저장 시와 전송 시 모두 암호화됩니다. 파일은 Azure OpenAI 리소스 내에서만 접근 가능합니다.

In [5]:
# Create Azure OpenAI Client

import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=get_bearer_token_provider(
        DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

In [6]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
# Note: You can visit the Azure AI Foundry Portal - and look under your Azure AI Project's 'Data Files' tab to see the uploaded files.

training_response = client.files.create(
    file=open(training_file, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-4a82006f7b6544f2aeadabe128900e4d
Validation file ID: file-9db0bcbf55cd4d709fc274d4251c703b


## Step 5: 미세 조정 작업 제출

업로드한 데이터셋을 사용하여 미세 조정 작업을 생성하고 제출하세요. 주요 매개변수는 다음과 같습니다.

- **기본 모델**: `gpt-4o-2024-08-06` (Fine-Tuning 호환 버전)
- **학습 파일**: 업로드한 학습 데이터셋
- **검증 파일**: 업로드한 검증 데이터셋
- **시드**: 재현 가능한 결과를 위해 (선택 사항이지만 권장)

> **중요**: 미세 조정 작업은 데이터셋 크기에 따라 일반적으로 10~30분 정도 소요됩니다. 작업은 자동으로 학습 체크포인트를 생성하고 검증 손실을 모니터링합니다.

In [7]:
# Submit fine-tuning training job
# Add a delay so the cell runs for about 1 minute
import time
time.sleep(60)

# Note that the model you specify here must be one that can be fine-tuned.
# Currently gpt-4o can be fine-tuned only in Sweden Central and North Central US
# See: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/fine-tuning?tabs=azure-openai&pivots=programming-language-python#prerequisites
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters.
    model="gpt-4o-2024-08-06",
    # seed parameter controls reproducibility of the fine-tuning job. If no seed is specified one will be generated automatically.
    seed=105,
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.
print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

Job ID: ftjob-25dbaf49417c425e80f41e39b73a13d7
Status: ftjob-25dbaf49417c425e80f41e39b73a13d7
{
  "id": "ftjob-25dbaf49417c425e80f41e39b73a13d7",
  "created_at": 1767785734,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "batch_size": -1,
    "learning_rate_multiplier": 1.0,
    "n_epochs": -1
  },
  "model": "gpt-4o-2024-08-06",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 105,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-4a82006f7b6544f2aeadabe128900e4d",
  "validation_file": "file-9db0bcbf55cd4d709fc274d4251c703b",
  "estimated_finish": 1767786636,
  "integrations": null,
  "metadata": null,
  "method": null
}


---

## Step 6: 미세 조정 작업 상태 추적

미세 조정 작업 진행 상황을 실시간으로 모니터링하세요. 작업은 다음 단계를 거칩니다.

1. **검증 중**: 데이터 형식 및 호환성 확인
2. **실행 중**: 데이터 세트를 사용한 학습 진행 중
3. **성공**: 학습이 성공적으로 완료됨
4. **실패**: 학습 중 오류 발생

> **모니터링**: 완료될 때까지 10초마다 셀이 자동으로 새로 고쳐집니다. Microsoft Foundry 포털에서도 진행 상황을 확인할 수 있습니다.
---

In [8]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-25dbaf49417c425e80f41e39b73a13d7 finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 1 fine-tune jobs.


---

## Step 7: 미세 조정 이벤트 검토

자세한 학습 이벤트 및 로그를 검토하여 학습 과정을 이해합니다.

- **학습 진행 상황**: 단계별 학습 업데이트
- **손실 지표**: 학습 및 검증 손실 변화
- **완료 상태**: 최종 학습 결과 및 모델 위치

> **디버깅**: 이벤트를 통해 학습 문제를 해결하고 학습 완료 여부를 확인할 수 있습니다.

In [9]:
response = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=job_id, limit=10)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftevent-b0958a69929b49fa86dbdf982d341d04",
      "created_at": 1767791214,
      "level": "info",
      "message": "Training tokens billed: 8000",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-fb8130f79a9f43499bd0988e0a47af80",
      "created_at": 1767791214,
      "level": "info",
      "message": "Model Evaluation Passed.",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-be400abf92fa415e9d98e1b5579f2838",
      "created_at": 1767791214,
      "level": "info",
      "message": "Completed results file: file-69eac3f2e8d24d279ef14d116c3296c3",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-5a8c31c7b53b4943aac0bb6a104d02c2",
      "created_at": 1767791205,
      "level": "info",
      "message": "Job succeeded.",
      "object": "fine_tuning.jo

---

## Step 8: 미세 조정 체크포인트 검토

미세 조정 과정에서 생성된 학습 체크포인트를 검토합니다.

- **체크포인트 분석**: 다양한 학습 단계에서의 모델 상태
- **성능 지표**: 각 체크포인트에서의 검증 손실
- **모델 선택**: 가장 성능이 좋은 체크포인트 식별

> **고급 활용**: 체크포인트를 통해 최적의 학습 종료 시점을 선택하고 학습 과정을 분석할 수 있습니다.

In [10]:
response = client.fine_tuning.jobs.checkpoints.list(job_id)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftchkpt-cfbba835f11e46bb8b6cd53026028762",
      "created_at": 1767789935,
      "fine_tuned_model_checkpoint": "gpt-4o-2024-08-06.ft-25dbaf49417c425e80f41e39b73a13d7",
      "fine_tuning_job_id": "ftjob-25dbaf49417c425e80f41e39b73a13d7",
      "metrics": {
        "full_valid_loss": 1.5599855437423245,
        "full_valid_mean_token_accuracy": 0.5984848484848485,
        "step": 120.0,
        "train_loss": 1.3127870559692383,
        "train_mean_token_accuracy": 0.6190476190476191,
        "valid_loss": 1.3850951385498047,
        "valid_mean_token_accuracy": 0.64
      },
      "object": "fine_tuning.job.checkpoint",
      "step_number": 120
    },
    {
      "id": "ftchkpt-d43109715cdb44bab5904c13ea48ab94",
      "created_at": 1767789368,
      "fine_tuned_model_checkpoint": "gpt-4o-2024-08-06.ft-25dbaf49417c425e80f41e39b73a13d7:ckpt-step-80",
      "fine_tuning_job_id": "ftjob-25dbaf49417c425e80f41e39b73a13d7",
      "metrics": {
        "full_val

---

## Step 9: 미세 조정된 모델 이름 가져오기

배포에 사용할 미세 조정된 모델 식별자를 가져옵니다.

- **모델 ID**: 사용자 지정 모델의 고유 식별자
- **학습 통계**: 최종 학습 지표 및 완료 세부 정보
- **배포 준비 완료**: 모델이 Azure에 배포할 준비가 되었습니다.

> **다음 단계**: 이 모델 ID를 사용하여 Azure AI Foundry 포털에서 배포를 생성합니다.

In [11]:
# Retrieve fine_tuned_model name

response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

{
  "id": "ftjob-25dbaf49417c425e80f41e39b73a13d7",
  "created_at": 1767785734,
  "error": null,
  "fine_tuned_model": "gpt-4o-2024-08-06.ft-25dbaf49417c425e80f41e39b73a13d7",
  "finished_at": 1767791214,
  "hyperparameters": {
    "batch_size": 1,
    "learning_rate_multiplier": 1.0,
    "n_epochs": 3
  },
  "model": "gpt-4o-2024-08-06",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-69eac3f2e8d24d279ef14d116c3296c3"
  ],
  "seed": 105,
  "status": "succeeded",
  "trained_tokens": 10485,
  "training_file": "file-4a82006f7b6544f2aeadabe128900e4d",
  "validation_file": "file-9db0bcbf55cd4d709fc274d4251c703b",
  "estimated_finish": 1767787613,
  "integrations": null,
  "metadata": null,
  "method": null
}


---

## Step 10: 테스트를 위해 미세 조정된 모델 배포

개선된 어조와 스타일을 테스트하기 위해 미세 조정된 모델을 배포합니다.

1. **Microsoft Foundry Portal**: 프로젝트의 모델 배포로 이동합니다.
2. **사용자 지정 모델 배포**: 미세 조정된 모델 ID를 선택합니다.
3. **개발자 등급**: 테스트용으로 사용합니다(추론 비용만 발생하며 호스팅 비용은 없습니다).
4. **배포 구성**: 배포 이름과 리소스 할당을 설정합니다.

> **비용 최적화**: 개발자 등급을 사용하면 호스팅 비용 없이 테스트할 수 있습니다. 프로덕션 환경에서 사용하려면 표준 배포로 업그레이드하세요.

> Prompt 1: What kind of paint should I buy for my outdoor deck?

```txt
🪵 Deck protection options! Semi-Transparent Deck Stain at 38 enhances wood grain, or Deck & Fence Stain at 36 for UV protection?
```

> Prompt 2: I'm painting over rust - what spray paint should I use?

👍 Right choice! Rust Prevention Spray at $13 applies directly over rust with long-lasting protection. Primer recommendation?

## 🎯 미세 조정 결과 및 인사이트

**Zava 톤 일관성 확보!**

위 두 예시에서, 미세 조정된 모델은 이제 Zava의 브랜드 가이드라인인 "정중하고, 사실적이며, 도움이 되는" 응답을 일관되게 따릅니다.

### ✅ 일관된 구조
- **이모티콘으로 시작**: 모든 응답은 관련 이모지로 시작합니다.
- **정중한 인정**: 첫 문장에서 고객의 요구 사항을 인정합니다.
- **사실 정보**: 중간 부분에서 구체적인 제품 세부 정보와 가격을 제공합니다.
- **도움이 되는 후속 조치**: 마지막 문장에서 추가적인 도움을 제공합니다.

### 🚀 주요 이점
- **짧은 프롬프트**: 모든 요청에 ​​대해 몇 개의 예시를 반복할 필요가 없습니다.
- **토큰 비용 절감**: 프롬프트 길이가 짧아져 API 비용이 절감됩니다.
- **처리 속도 향상**: 처리해야 할 컨텍스트가 줄어들어 응답 속도가 빨라집니다.
- **일관된 품질**: 모든 응답이 학습된 패턴을 따릅니다.

### 📊 성능 비교
| 측면 | 미세 조정 전 | 미세 조정 후 |
--------|--------------------|--------------------|
| 프롬프트 길이 | 약 800 토큰 (예제 포함) | 약 200 토큰 (시스템 프롬프트만) |
| 어조 일관성 | 가변적 (예제에 따라 다름) | 일관적 (모델에 내장) |
| 응답 시간 | 느림 (프롬프트가 김) | 빠름 (프롬프트가 짧음) |
| 요청당 비용 | 높음 (입력 토큰이 많음) | 낮음 (입력 토큰이 적음) |

> **모범 사례**: 미세 조정은 단순히 사실적 지식(RAG에서 더 잘 처리됨)을 제공하는 것보다 모델이 학습해야 할 일관된 패턴이 있을 때 가장 효과적입니다.

## Step 11: 다음 단계

모델의 어조와 스타일을 성공적으로 미세 조정하셨습니다! 다음 단계를 따라 진행하세요.

> **Key Insight**: 미세 조정은 어조와 스타일과 같은 일관된 패턴을 내재화하는 데 탁월하며, RAG는 최신 사실 정보를 제공하는 데 탁월합니다. 이 둘을 결합하면 강력하고 비용 효율적인 AI 시스템을 구축할 수 있습니다.

---

**Great work! You've mastered fine-tuning for tone and style customization.** 🎉